In [1]:
%reset -f
import sys
sys.path.insert(0,'..')
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Reshape
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle
seed = 2018
np.random.seed(seed)
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.utils import np_utils
from utils import TEMP_DATADIR
from wavhandler import *
from sklearn.utils import shuffle
from utils_train import *

setting = 'gasf'
model_name = 'autoencoder'
undersampling = True
dataset = Dataset('LG')
dataset.load(only_names=True, nr_signals=np.inf)
X_names = dataset.filenames
y = dataset.y

# Training setting - What kind of data to use
if setting in ['gasf','gadf', 'mtf', 'rp']:
    input_shape = (148,148,1)
elif setting=='stft':
    input_shape = (129, 120, 1)

# More settings
model_name = model_name + '_' + setting + '_'
top_weights_path = TEMP_DATADIR + str(model_name) + '.h5'
logfile = TEMP_DATADIR + str(model_name) + '.log'
batch_size = 32
monitor = 'val_loss'
es_patience = 7
rlr_patience = 3
target_names = np.unique(y)

y = LabelEncoder().fit_transform(y)
X = np.array(X_names).reshape(-1,1)

X, y = shuffle(X, y, random_state=0) 

if undersampling:
    from imblearn.under_sampling import RandomUnderSampler
    ros = RandomUnderSampler(random_state=0)
    ros.fit(X,y)
    X, y = ros.fit_resample(X,y)
    X = pd.Series(X.ravel()).tolist()
    print('After undersampling: \n{}\n'.format(pd.DataFrame(y).iloc[:,0].value_counts()))
else:
    print('Class balance: \n{}\n'.format(pd.DataFrame(y).iloc[:,0].value_counts()))


X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.10, 
                                                    stratify=y, 
                                                    random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                test_size=0.2, 
                                                stratify=y_train,  
                                                random_state=0)
print("Train shape: \t{}, \nTest shape: \t{}, \nValid shape: \t{}".format(len(X_train), len(X_test), len(X_val)))

callbacks_list = [ModelCheckpoint(monitor = monitor,
                                filepath = top_weights_path,
                                save_best_only = True,
                                save_weights_only = True,
                                verbose = 1),
                    EarlyStopping(monitor = monitor,
                                patience = es_patience,
                                verbose = 1),
                    ReduceLROnPlateau(monitor = monitor,
                                factor = 0.1,
                                patience = rlr_patience,
                                verbose = 1),
                    CSVLogger(filename = logfile)]

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.
100%|██████████| 2/2 [00:00<00:00, 35.33it/s]

After undersampling: 
1    5536
0    5536
Name: 0, dtype: int64

Train shape: 	7971, 
Test shape: 	1108, 
Valid shape: 	1993


In [2]:
print('input shape:', input_shape)
# Encoder Layers
model = Sequential()
model.add(Conv2D(64, 3, activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(2))
model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(2))
model.add(Conv2D(1, 3, activation='relu', padding='same'))
model.add(UpSampling2D(2))
model.add(Conv2D(32, 3, activation='relu', padding='same'))
model.add(UpSampling2D(2))
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(Conv2D(1, 3, activation='relu', padding='same'))
model.compile(optimizer='adam', loss='mse')
model.summary()

input shape: (148, 148, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 74, 74, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 1)         289       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 74, 74, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 74, 74, 32)  

In [3]:
# TRAIN
model.fit_generator(train_generator2(X_train,
                                    batch_size=batch_size, 
                                    target_names=target_names,
                                    setting=setting),
                    steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                    epochs=100, 
                    validation_data = valid_generator2(X_val,
                                                    batch_size=batch_size, 
                                                    target_names=target_names,
                                                    setting=setting), 
                    validation_steps = int(math.ceil(float(len(X_test)) / float(batch_size))),
                    callbacks = callbacks_list)

Epoch 1/100
250/250 [==============================] - 70s 279ms/step - loss: 0.5902 - val_loss: 0.5847

Epoch 00001: val_loss improved from inf to 0.58465, saving model to /home/kalfasyan/projects/wingbeat_frequencies/temp_data/autoencoder_gasf_.h5
Epoch 2/100
250/250 [==============================] - 63s 250ms/step - loss: 0.5902 - val_loss: 0.5890

Epoch 00002: val_loss did not improve from 0.58465
Epoch 3/100
250/250 [==============================] - 62s 250ms/step - loss: 0.5902 - val_loss: 0.5912

Epoch 00003: val_loss did not improve from 0.58465
Epoch 4/100
250/250 [==============================] - 63s 251ms/step - loss: 0.5902 - val_loss: 0.5821

Epoch 00004: val_loss improved from 0.58465 to 0.58207, saving model to /home/kalfasyan/projects/wingbeat_frequencies/temp_data/autoencoder_gasf_.h5
Epoch 5/100
250/250 [==============================] - 62s 249ms/step - loss: 0.5902 - val_loss: 0.5922

Epoch 00005: val_loss did not improve from 0.58207
Epoch 6/100
250/250 [=======

In [ ]:
# open a terminal and start TensorBoard to read logs in the autoencoder subdirectory
# tensorboard --logdir=autoencoder

# autoencoder.fit(x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(x_test, x_test),
#                 callbacks=[TensorBoard(log_dir='conv_autoencoder')], verbose=2)

# take a look at the reconstructed digits
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(10, 4), dpi=100)
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.set_axis_off()

    # display reconstruction
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.set_axis_off()

plt.show()

# take a look at the 128-dimensional encoded representation
# these representations are 8x4x4, so we reshape them to 4x32 in order to be able to display them as grayscale images

encoder = Model(input_img, encoded)
encoded_imgs = encoder.predict(x_test)

# save latent space features 128-d vector
pickle.dump(encoded_imgs, open('conv_autoe_features.pickle', 'wb'))

n = 10
plt.figure(figsize=(10, 4), dpi=100)
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(encoded_imgs[i].reshape(4, 4 * 8).T)
    plt.gray()
    ax.set_axis_off()

plt.show()

K.clear_session()